In [0]:
from pyspark.sql.functions import col
user_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
file_location = f'/Workspace/Users/{user_name}/data/telco.csv'
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

from pyspark.sql.functions import trim, when 
#df_tc_dbl = df.withColumn('TotalCharges', trim(col("TotalCharges")))

df_tc_dbl = df.withColumn("TotalCharges", when(col('TotalCharges')==" " , None) \
          .otherwise(col('TotalCharges')))
df_tc_dbl = df_tc_dbl.withColumn('TotalCharges', col('TotalCharges').cast('double'))

display(df_tc_dbl)

In [0]:
df_selection = df_tc_dbl.select('Contract', 'PhoneService', 'InternetService', 'TotalCharges', 'MonthlyCharges', 'tenure')
#display(df_selection)
#df_filtered = df_selection.filter( \
#    (df_selection['Contract'] == 'One year') & (df_selection['MonthlyCharges'] > 20))

df_filtered = df_selection.filter( \
    ((df_selection['Contract'] == 'One year') | (df_selection['Contract'] == 'Two year')) & (df_selection['MonthlyCharges'] > 100))

df_filtered = df_filtered.sort(df_filtered['MonthlyCharges'].desc())

from pyspark.sql.functions import desc
df_filtered = df_filtered.sort(desc(df_filtered['MonthlyCharges']))
#df_filtered = df_filtered.sort(df_filtered['MonthlyCharges'].asc())

display(df_filtered)